# CLEAN AND STACK

In [ ]:
from file_utils import FileManager

fm = FileManager()                
df1 = fm.get_csv("/Users/intankwardhani/Documents/BeCode/immo-eliza-team-horses-analysis/data/properties_all_provinces.csv")
df2 = fm.get_csv("/Users/intankwardhani/Documents/BeCode/immo-eliza-team-horses-analysis/data/properties_data2.csv")

# sort the columns alphabetically
df1 = df1[sorted(df1.columns)]
df2 = df2[sorted(df2.columns)]

In [ ]:
# check column names
list_colnames_df1 = df1.columns.tolist()
list_colnames_df2 = df2.columns.tolist()

print(list_colnames_df1)
print(list_colnames_df2)

In [ ]:
# rename some columns for uniformity
df1 = df1.rename(columns={
    "url": "property_url"
})

df2 = df2.rename(columns={
    "facade": "facades",
    "living_surface": "living_area"
})

print(df1.columns.tolist())
print(df2.columns.tolist())


In [ ]:
# remove columns we don't need
df1cols_todrop = ['equipped_kitchen', 'furnished', 'garden_area']
df2cols_todrop = ['accessibility', 'address', 'elevator', 'floor', 'floor_heating',
                  'furnished', 'garage', 'glazing', 'number_baths', 'number_garage', 'project_url']

ddf1 = df1.drop(columns=df1cols_todrop)
ddf2 = df2.drop(columns=df2cols_todrop)

# reorder the colnames in the new dfs
ddf1 = ddf1[sorted(ddf1.columns)]
ddf2 = ddf2[sorted(ddf2.columns)]

print(ddf1.columns.tolist())
print(ddf2.columns.tolist())

# check the number of columns and rows
print(ddf1.shape[0])
print(ddf2.shape[0])
print(ddf1.shape[1])
print(ddf2.shape[1])

In [ ]:
# clean and normalise data: float --> int, numerical string --> int, pure string --> str

from data_utils import DataCleaner

# cols_toclean = ['build_year', 'facades', 'garden', 'living_area', 'number_rooms', 'postal_code',
#                 'swimming_pool', 'terrace']
cleaner = DataCleaner()
clean_df1 = cleaner.extract_int_from_string(ddf1, columns=['price'])
clean_df2 = cleaner.float_to_int(ddf2, columns=['number_rooms', 'postal_code', 'price'])

In [ ]:
# show
clean_df1.head(20)

In [ ]:
# clean and normalise data: float --> int, numerical string --> int, pure string --> str

from data_utils import DataCleaner

cols_toclean = ['build_year', 'facades', 'garden', 'living_area', 'number_rooms', 'postal_code',
                'swimming_pool', 'terrace']
cleaner = DataCleaner()
clean_df1 = cleaner.float_to_int(ddf1, columns=cols_toclean)
clean_df2 = cleaner.float_to_int(ddf2, columns=cols_toclean)
clean_df1 = cleaner.extract_int_from_string(clean_df1, columns=['price'])
clean_df2 = cleaner.extract_int_from_string(clean_df2, columns=['price'])

In [ ]:
# show
clean_df2.head(20)

In [ ]:
print(clean_df1.shape[0])
print(clean_df2.shape[0])
print(clean_df1.shape[1])
print(clean_df2.shape[1])

In [ ]:
# finally stack the dataframes vertically together 
import pandas as pd
df = pd.concat([clean_df1, clean_df2], ignore_index=True)

In [ ]:
print(df.shape[0])
print(df.shape[1])

In [ ]:
# export the dataframe to csv
fm.export_csv(df, "/Users/intankwardhani/Documents/BeCode/immo-eliza-team-horses-analysis/data/properties_data_final.csv")

## Province Postal Code Range

1. Brussels Capital Region    1000–1299  
2. Antwerp    2000–2990  
3. Flemish Brabant    1500–1999, 3000–3499  
4. Walloon Brabant    1300–1499  
5. East Flanders    9000–9999
6. West Flanders    8000–8999 (e.g., Bruges is 8000)
7. Hainaut    6000–6599, 7000–7999
8. Liège    4000–4999
9. Limburg    3500–3990
10. Luxembourg    6000-6599, 66xx-69xx, 50xx-56xx (some overlap with Hainaut and Namur)
11. Namur    5000-5999 (some overlap with Luxembourg)

# VISUALISE

Use data from ```final_clean.csv``` only!

In [ ]:
from file_utils import FileManager

fm = FileManager()                
df = fm.get_csv('/Users/intankwardhani/Documents/BeCode/immo-eliza-team-horses-analysis/data/final_clean.csv')
# df.head(5)

## Categorise Build Year

In [ ]:
from data_utils import DataCleaner

cleaner = DataCleaner()
df = cleaner.categorise_build_years(df, year_col="build_year")
df = df[sorted(df.columns)]
df.head(5)


In [ ]:
import pandas as pd

df = df[df['living_area'].notna()]
df['living_area'] = pd.to_numeric(df['living_area'], errors='coerce')
df['living_area'] = df['living_area'].clip(0, 500)  # cap at 500 m²

mean_area = df['living_area'].mean()
std_area = df['living_area'].std()

print("Mean living area:", mean_area)
print("Standard deviation:", std_area)

In [ ]:
df["state"].unique()

In [ ]:
import importlib
import data_utils
importlib.reload(data_utils)
from data_utils import DataStats

ds = DataStats()
coef_df = ds.find_predictor_coeff(df)
coef_df

## Prediction Model with Total Coeff Values

In [ ]:
import importlib
import data_utils
importlib.reload(data_utils)
from data_utils import DataStats

ds = DataStats()
results = ds.analyze_predictor_importance(df)

results["variable_importance"]
results["performance"]
results["feature_level_coeffs"]


In [ ]:
a = results["feature_level_coeffs"]
b = a[a['variable'] == "terrace"]
sum(b['abs_coeff'])

## Percentage Price Change

In [ ]:
import matplotlib.pyplot as plt

# Top 4 variables and their % price change
variables = ["state", "province", "facades", "living_area"]
pct_change = [21.9, 16.9, 3.7, 4.08]  # from previous calculations

# Create vertical bar plot
plt.figure(figsize=(8, 5))
bars = plt.bar(variables, pct_change, color='pink')
plt.ylabel("% Price Change")
plt.xlabel("Variable")
plt.title("Approximate % Price Change per Unit/Category")

# Add value labels on top of bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, height + 0.5,
             f"{height:.2f}%", ha='center', va='bottom')

plt.tight_layout()
plt.show()


## Compare for-sale counts

In [ ]:
# compare for-sale numbers between brussels and namur
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Example pastel colors
brussels_colour = "#f7c6d0"   # pastel pink
namur_colour    = "#d7c6f7"   # pastel purple
eastflanders_colour = "#fff068" 
liege_colour = "#91e8e3"


# ---- FILTER THE DATA ----
brussels = df[df["province"] == "Brussels Capital Region"]
namur = df[df["province"] == "Namur"]
east_flanders = df[df["province"] == "East Flanders"]
liege = df[df["province"] == "Liège"] 

# ---- COUNT BY PROPERTY TYPE ----
counts_brussels = brussels["property_type"].value_counts()
counts_namur = namur["property_type"].value_counts()
counts_eastflanders = east_flanders["property_type"].value_counts()
counts_liege = liege["property_type"].value_counts()

# ---- FIND SHARED & UNIQUE PROPERTY TYPES ----
shared_types = list(set(counts_brussels.index) & set(counts_namur.index))
unique_brussels = list(set(counts_brussels.index) - set(counts_namur.index))
unique_namur = list(set(counts_namur.index) - set(counts_brussels.index))
shared_types2 = list(set(counts_eastflanders.index) & set(counts_liege.index))
unique_eastflanders = list(set(counts_eastflanders.index) - set(counts_liege.index))
unique_liege = list(set(counts_liege.index) - set(counts_eastflanders.index))

# ---- CREATE DATAFRAMES FOR PLOTTING ----

# Shared types
shared_df = pd.DataFrame({
    "property_type": shared_types,
    "Brussels": counts_brussels[shared_types].values,
    "Namur": counts_namur[shared_types].values
})

shared_df2 = pd.DataFrame({
    "property_type": shared_types2,
    "East Flanders": counts_eastflanders[shared_types2].values,
    "Liège": counts_liege[shared_types2].values
})

# Melt for seaborn
shared_melt = shared_df.melt(
    id_vars="property_type",
    value_vars=["Brussels", "Namur"],
    var_name="Region",
    value_name="Count"
)

shared_melt2 = shared_df2.melt(
    id_vars="property_type",
    value_vars=["East Flanders", "Liège"],
    var_name="Region",
    value_name="Count"
)

# Unique types
unique_list = []
for t in unique_brussels:
    unique_list.append(["Brussels", t, counts_brussels[t]])
for t in unique_namur:
    unique_list.append(["Namur", t, counts_namur[t]])

unique_df = pd.DataFrame(unique_list, columns=["Region", "property_type", "Count"])

unique_list2 = []
for t in unique_eastflanders:
    unique_list2.append(["East Flanders", t, counts_eastflanders[t]])
for t in unique_liege:
    unique_list2.append(["Liège", t, counts_liege[t]])

unique_df2 = pd.DataFrame(unique_list2, columns=["Region", "property_type", "Count"])

# ---- PLOT 1: SHARED PROPERTY TYPES ----
plt.figure(figsize=(10, 6))
sns.barplot(
    data=shared_melt,
    x="property_type",
    y="Count",
    hue="Region",
    palette=[brussels_colour, namur_colour]
)
plt.title("Shared Property Types: Brussels vs Namur")
plt.xlabel("Property Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ---- PLOT 2: UNIQUE PROPERTY TYPES ----
plt.figure(figsize=(10, 6))
sns.barplot(
    data=unique_df,
    x="property_type",
    y="Count",
    hue="Region",
    palette=[brussels_colour, namur_colour]
)
plt.title("Unique Property Types in Brussels and Namur")
plt.xlabel("Property Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ---- PLOT 3: SHARED PROPERTY TYPES 2 ----
plt.figure(figsize=(10, 6))
sns.barplot(
    data=shared_melt2,
    x="property_type",
    y="Count",
    hue="Region",
    palette=[eastflanders_colour, liege_colour]
)
plt.title("Shared Property Types: East Flanders vs Liège")
plt.xlabel("Property Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# ---- PLOT 4: UNIQUE PROPERTY TYPES 2 ----
plt.figure(figsize=(10, 6))
sns.barplot(
    data=unique_df2,
    x="property_type",
    y="Count",
    hue="Region",
    palette=[eastflanders_colour, liege_colour]
)
plt.title("Unique Property Types in East Flanders and Liège")
plt.xlabel("Property Type")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# count by all property types each province has
brussels_all_types = brussels.shape[0]
namur_all_types = namur.shape[0]
eastflanders_all_types = east_flanders.shape[0]
liege_all_types = liege.shape[0]

# Labels, sizes, colors
labels = ["Brussels", "Namur", "East Flanders", "Liège"]
sizes = [brussels_all_types, namur_all_types, eastflanders_all_types, liege_all_types]
colours = [brussels_colour, namur_colour, eastflanders_colour, liege_colour]  # already defined earlier

# Plot
fig, ax = plt.subplots(figsize=(6, 6))
wedges, texts, autotexts = ax.pie(
    sizes,
    labels=labels,
    autopct="%1.1f%%",
    startangle=90,
    colors=colours,
    pctdistance=0.8
)

# Donut hole
center_circle = plt.Circle((0, 0), 0.45, color='white', fc='white')
ax.add_artist(center_circle)

plt.title("Property Count Comparison")
plt.tight_layout()
plt.show()


In [ ]:
provinces_sorted = sorted(df['province'].unique())